In [1]:
# Import Libraries
import numpy as np

from project_utils.bb_model import BB_Model
from project_utils.feature_stats import Feature_Statistics
from project_utils.feature_stats import Classes_Container


In [2]:
# Load Wine model

Wine_Data = BB_Model('Wine', mode = 'classification')

Wine_Data.df().head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
Wine_Data.MPL()

0.5102040816326531


MLPClassifier(random_state=RandomState(MT19937) at 0x2985DF81C40)

In [4]:
Wine_Data.Random_Forest()

0.6826530612244898


RandomForestClassifier(random_state=RandomState(MT19937) at 0x2985DF81C40)

In [5]:
print(Wine_Data.get_features())

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


In [6]:
#Default settings
from unravel.tabular import UnRAVELTabularExplainer

X_train, X_test, y_train, y_test = Wine_Data.get_TT_data()

#Initializing the explainer object
Unravel_Exp = UnRAVELTabularExplainer(bbox_model           = Wine_Data.get_Random_Forest(),
                                      train_data           = X_train,
                                      categorical_features = Wine_Data.get_catagorical(),
                                      mode                 = Wine_Data.get_mode())       
        


warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [7]:
# Generate explantion data for Random Forest

Wine_Stats = Feature_Statistics(feature_names = Wine_Data.get_features(), mode='classification', classes=Wine_Data.get_classes())

for test_instance in range(20):#X_test.shape[0]):

    print('INSTANCE: ', test_instance)
    
    data_row  = np.array([X_test[test_instance]])
    
    explain_inst = Unravel_Exp.explain(X_init            = data_row,
                                       feature_names     = Wine_Data.get_features(),
                                       alpha             = "FUR",
                                       max_iter          = 20,
                                       normalize         = False,
                                       importance_method = "ARD")
           
    f_predict = Wine_Data.get_Random_Forest().predict_proba(data_row.reshape(1, -1))

    e_predict = Unravel_Exp.gpmodel_predict(data_row)
    
    e_predict = np.array(e_predict).ravel()
    print(e_predict)

    Wine_Stats.Add_Sample(sample       = data_row,
                          outcome      = y_test[test_instance],
                          f_prediction = f_predict)


INSTANCE:  0
[-1.98676044  0.07854352]


<class 'ValueError'>: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 7 and the array at index 1 has size 2

In [ ]:
Wine_Stats.Feature_Counts(max_features=8, scaled=True, threshold=0.075)

In [ ]:
Wine_Stats.Frequency_Plot()

In [ ]:
Wine_Stats.Box_Plot()

In [ ]:
Wine_Stats.Violin_Plot(top_features=False)

In [ ]:
Wine_Container = Classes_Container(Wine_Stats)

Wine_Container.Feature_Counts(max_features=11, scaled=True, threshold=0.1)


In [ ]:
Wine_Container.Frequency_Plot()

In [ ]:
Wine_Stats.Fidelity()

In [ ]:
Wine_Stats.Jaccard_Values()

In [ ]:
Wine_Stats.Class_Fidelity_Graph()